AHP层次分析法
1.确定方案、指标
2.为指标权重、各方案在某指标下的权重构建判断矩阵
3.一致性检验，调整权重（代码实现）
4.选择合适的方案计算权重（代码实现）
    可分为3钟：算数平均法、几何平均法、特征值法
5.汇总
6.求和

In [1]:
#导入所需库
import numpy as np

准备工作：数据预处理(此处为输入判断矩阵)

In [ ]:
# 定义矩阵A，np.array 是 numpy 库中的一个函数，用于创建数组。
# 它将输入的对象(如列表、元组、其他数组等)转换为 numpy 数组，用于矩阵计算
# 原判断矩阵
A = np.array([[1,2,3,5],
              [1/2,1,1/2,2],
              [1/3,2,1,2],
              [1/5,1/2,1/2,1]]) # 修改后的判断矩阵


第一步：一致性检验
1.计算CI
2.根据n(维度)查找RI
3.计算一致性比例CR，当CR<0.1时，表示接受

In [8]:
# 获取A的行，A.shape返回一个元组，表示数组的维度[行数, 列数]
n = A.shape[0] #获取矩阵的行数，即维度

# 求出最大特征值以及对应的特征向量，np.linalg.eig是NumPy 库中的一个函数
# 用于计算一个方阵的特征值和特征向量。它返回两个数组
# 第一个是一维数组，包含全部特征值，第二个数组是二维数组，包含全部特征向量
eig_val,eig_vec = np.linalg.eig(A) # eig_val是特征值，eig_vec是特征向量
Maxeig = max(eig_val) # 求特征值的最大值
 
CI = (Maxeig-n)/(n-1)#计算一致性指标CI
RI = [0,0.0001,0.52,0.89,1.12,1.26,1.36,1.41,1.46,1.49,1.52,1.54,1.56,1.58, 1.59] #平均随机一致性指标RI
# 注意，这里的RI最多支持n=15
# 这里n=2时，一定是一致矩阵，所以CI =0，我们为了避免分母为0，将这里的第二个元素改为了很接近0的正数
 
CR = CI/RI[n-1]#计算一致性比例CR
print('一致性指标CI =',CI)
print('一致性比例CR =',CR)
if CR< 0.10:
    print('因为CR<0.10,所以该判断矩阵A的一致性可以接受!')
else:
    print('注意:CR>= 0.10,因此该判断矩阵A需要进行修改!')

一致性指标CI = (0.03761001273071566+0j)
一致性比例CR = (0.04225844127046704+0j)
因为CR<0.10,所以该判断矩阵A的一致性可以接受!


第二步：计算权重（共三种方法）

第一种：算术平均法求权重

In [ ]:
# 1.计算每列的和
# np.sum 函数可以计算一维数组中所有元素的总和。
# 还可以通过指定 axis 參数来计算多维数组的某个维度上的元素总和。
# 例如，在二维数组中，axis=0 表示按列计算总和，axis=1 表示按行计算总和。
ASum = np.sum(A, axis=0)

# 获取A的行
n = A.shape[0]

# 2.归一化
#二维数组除以一维数组，会自动将一维数组扩展为与二维数组相同的形状，然后进行逐元素的除法运算。(广播机制)
stand_A = A/ASum

#3.算术平均法
# 各列相加到同一行，axis=1表示按行计算总和
ASumr = np.sum(stand_A, axis=1)
#计算权重向量
weights = ASumr/n
print('算术平均法求的权重结果为：',weights)

算术平均法求的权重结果为： [0.48885991 0.18192996 0.2318927  0.09731744]


第二种：几何平均法求权重

In [10]:
# 获取A的行
n = A.shape[0]

# 1.将A中按行相乘得到一列向量
# np.prod 函数可以计算一维数组中所有元素的乘积。
# 还可以通过指定 axis 參数来计算多维数组的某个维度上的元素柔积。
# 例如，在二维数组中，axis=0 表示按列计算乘积，axis=1 表示按行计算乘积。
prod_A = np.prod(A, axis=1)

# 2.将新的向量的每个分量开n次方等价求1/n次方
# np.power是NumPy库中的一个函数，用于对数组中的元素进行幂运算。
# 例如，可以使用np.power(a，b)对数组a中的每个元素都按照b指数进行运算。
prod_n_A=np.power(prod_A, 1/n)

# 3.归一化处理
re_prod_A= prod_n_A/ np.sum(prod_n_A)
# 打印权重结果
print('几何平均法求的权重结果为：',re_prod_A)

几何平均法求的权重结果为： [0.49492567 0.17782883 0.22724501 0.1000005 ]


第三种：特征值法求权重（当CR足够小时，可认为判断矩阵为一致矩阵）

In [ ]:
# 获取A的行
n = A.shape[0]

#1.求出最大特征值及其对应的特征向量
# 求出特征值和特征向量（注意，特征向量是列向量，每一个特征值对应一个特征向量）
eig_values,eig_vectors = np.linalg.eig(A)
# 找出最大特征值的索引，np.argmax是numPy库中的一个函数，用于返回数组中最大值的索引
max_index = np.argmax(eig_values)
# 找出对应的特征向量
max_vector = eig_vectors[:, max_index]

#2.对特征向量进行归一化处理,得到权重
weights = max_vector /np.sum(max_vector)
# 输出权重
print('特征值法求的权重结果为：',weights)